In [1]:
import torch
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
%matplotlib inline
import datetime
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

cifar10 = datasets.CIFAR10(
    '.', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    '.', train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

Files already downloaded and verified
Files already downloaded and verified


# 1. Kernerl_size=5

In [3]:
5 // 2

2

In [4]:

class Net(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, 16, self.kernel_size, padding=self.kernel_size // 2)
        self.conv2 = nn.Conv2d(16, 8, self.kernel_size, padding=self.kernel_size // 2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)
        
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out


def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    device = ( torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
    for epoch in range(1, n_epochs + 1):  
        loss_train = 0.0
        for imgs, labels in train_loader:  
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            
            outputs = model(imgs) 
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()  
            loss.backward()  
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))
            
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) 
                total += labels.shape[0] 
                correct += int((predicted == labels).sum()) 

        print("Accuracy {}: {:.2f}".format(name , correct / total))

In [5]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

### the number of parameters

In [6]:
device = ( torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f'My device is {device}')

Net_conv3 = Net(3).to(device=device)
Net_conv5 = Net(5).to(device=device)

numel_list3 = [p.numel() for p in Net_conv3.parameters()]
print(f'kernel_size = 3 : {numel_list3} sum : {sum(numel_list3)}')

numel_list5 = [p.numel() for p in Net_conv5.parameters()]
print(f'kernel_size = 5 : {numel_list5} sum : {sum(numel_list5)}')

My device is cuda
kernel_size = 3 : [432, 16, 1152, 8, 16384, 32, 64, 2] sum : 18090
kernel_size = 5 : [1200, 16, 3200, 8, 16384, 32, 64, 2] sum : 20906


### kernel_size = 3

In [7]:
n_epochs = 100
optimizer = optim.SGD(Net_conv3.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs, optimizer, Net_conv3, loss_fn, train_loader)
validate(Net_conv3, train_loader, val_loader)

/home/takakiyuto/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


2021-09-15 14:34:53.425001 Epoch 1, Training loss 0.5798977567891407
2021-09-15 14:34:55.671588 Epoch 10, Training loss 0.33098394399995257
2021-09-15 14:34:58.177785 Epoch 20, Training loss 0.29061039712778325
2021-09-15 14:35:00.681818 Epoch 30, Training loss 0.2666620386254256
2021-09-15 14:35:03.191580 Epoch 40, Training loss 0.2483056040516325
2021-09-15 14:35:05.695692 Epoch 50, Training loss 0.23055490476500456
2021-09-15 14:35:08.206986 Epoch 60, Training loss 0.21382714565961983
2021-09-15 14:35:10.712608 Epoch 70, Training loss 0.19941808980931142
2021-09-15 14:35:13.224687 Epoch 80, Training loss 0.18296448708434773
2021-09-15 14:35:15.705480 Epoch 90, Training loss 0.16896311440475428
2021-09-15 14:35:18.194209 Epoch 100, Training loss 0.15702229638578027
Accuracy train: 0.94
Accuracy val: 0.89


In [8]:
n_epochs = 100
optimizer = optim.SGD(Net_conv5.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs, optimizer, Net_conv5, loss_fn, train_loader)
validate(Net_conv5, train_loader, val_loader)

2021-09-15 14:35:18.566027 Epoch 1, Training loss 0.5596386515031195
2021-09-15 14:35:20.925270 Epoch 10, Training loss 0.3280355742402897
2021-09-15 14:35:23.546714 Epoch 20, Training loss 0.282826970859318
2021-09-15 14:35:26.171057 Epoch 30, Training loss 0.25223569230289217
2021-09-15 14:35:28.802554 Epoch 40, Training loss 0.22613409209023616
2021-09-15 14:35:31.429111 Epoch 50, Training loss 0.20112430384963942
2021-09-15 14:35:34.058080 Epoch 60, Training loss 0.18068940676511472
2021-09-15 14:35:36.681772 Epoch 70, Training loss 0.1599704822537246
2021-09-15 14:35:39.314108 Epoch 80, Training loss 0.14135940394299046
2021-09-15 14:35:41.938980 Epoch 90, Training loss 0.1240192436298747
2021-09-15 14:35:44.561144 Epoch 100, Training loss 0.10829334141342503
Accuracy train: 0.97
Accuracy val: 0.88
